# Carga de archivos y librerías

In [ ]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# Cargar los archivos CSV en dos DataFrames
vuelos = pd.read_csv('Filghts TEC_Valid.csv')
ventas = pd.read_csv('Sales TEC_Valid.csv')

# Modificación y limpieza de datos

In [ ]:
# Solamente productos Perecedores
venta_freshfood = ventas[ventas['ProductType'] == 'Perecederos']
venta_freshfood.describe()

,Quantity,TotalSales
count,181990.000000,181990.000000
mean,2.488961,279.127276
std,1.880256,211.313158
min,1.000000,105.000000
25%,1.000000,115.000000
50%,2.000000,220.000000
75%,3.000000,345.000000
max,31.000000,3410.000000


In [ ]:
# Realizar join para unir los datos vuelo y la venta
venta_join = pd.merge(venta_freshfood, vuelos, on='Flight_ID', how='left')

In [ ]:
#Filtrar y limpiar los datos
venta_join_clean = venta_join.dropna(subset=['Passengers']) # sin info de pasajeros
# formato de tiempo
venta_join_clean['STD'] = pd.to_datetime(venta_join_clean['STD'], format = 'mixed')
venta_join_clean['STA'] = pd.to_datetime(venta_join_clean['STA'], format = 'mixed')
# datos solamente del 2023
datos_2023 = venta_join_clean.loc[venta_join_clean['STA'].dt.year == 2023]

<ipython-input-61-e584f0269441>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venta_join_clean['STD'] = pd.to_datetime(venta_join_clean['STD'], format = 'mixed')
<ipython-input-61-e584f0269441>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venta_join_clean['STA'] = pd.to_datetime(venta_join_clean['STA'], format = 'mixed')


# Modelo de predicción

## Manejo de variables (cualitativas y cuantitativas)

In [ ]:
# Definir transformador para codificar variables categóricas
categorical_features = ['ProductName', 'DepartureStation', 'ArrivalStation']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

# Definir transformador para escalar variables numéricas 
numeric_features = ['TotalSales', 'Capacity', 'Passengers']
numeric_transformer = Pipeline(steps=[
     ('scaler', StandardScaler())
 ])

# Combinar transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ])

## Función del modelo

In [ ]:
# Función para entrenar distintos modelos con distintos test_size y obtener su rendimiento
def lrm(datos_2023, model,i):
  # Dividir datos en características y etiquetas
  X = datos_2023.drop('Quantity', axis=1)
  y = datos_2023['Quantity']

  # Dividir datos en conjuntos de entrenamiento y prueba / con i% de prueba
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)

  # Entrenar el modelo
  model.fit(X_train, y_train)

  # Evaluar el modelo
  train_score = model.score(X_train, y_train)
  test_score = model.score(X_test, y_test)
  # Retornar las métricas
  return train_score, test_score

## Creación de modelos

In [ ]:
# Ciclo para obtener los rendimientos del modelo con distintos test-sizes
g = []
for i in [.2, .3, .4,.5]:
  # Crear el modelo de regresión lineal multivariable
  model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])
  g.append([lrm(datos_2023, model, i)])

## Resultados

Podemos observar que la diferencia no es significativa al aumentar el tamaño de test_size. Es por esto, que nuestro modelo para freshfood solamente consiste en una regresión lineal multivariada con un tamaño de test_size de 0.5 para que no usar tanto poder computacional a la hroa de entrenamiento.

In [ ]:
g

[[(0.9989060794841339, 0.9988822699165011)],
 [(0.9989049690231142, 0.998892703769858)],
 [(0.9989065906586336, 0.9988929840120807)],
 [(0.998903032126393, 0.9988993829317844)]]